In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import time
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
from PIL import Image

# ------------------------------
# Parameters
# ------------------------------
params = {
    'subset_ratio': 1.0,          # Adjust this to use a subset of the data (e.g., 0.1 for 10%)
    'num_epochs': 20,             # Number of epochs
    'batch_size': 32,             # Batch size
    'learning_rate': 1e-4,        # Learning rate for fine-tuning
    'weight_decay': 0.0,          # Weight decay (L2 regularization)
    'dropout_rate': 0.5,          # Dropout rate
    'num_classes': 397,           # Number of classes in SUN397
    'data_dir': '/Users/ryan/Projects/CV/Datasets/SUN397_flat',# Update this to your local SUN397 dataset path
    'save_dir': 'Figure Best 18 100pct'        # Directory to save figures and logs
}

# Create save directory
os.makedirs(params['save_dir'], exist_ok=True)

# ------------------------------
# Data Preparation
# ------------------------------
# Define transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
full_dataset = datasets.ImageFolder(params['data_dir'])
class_names = full_dataset.classes
print(f"Number of classes: {len(class_names)}")

# Create subset
subset_size = int(params['subset_ratio'] * len(full_dataset))
train_size = int(0.8 * subset_size)
val_size = subset_size - train_size
train_dataset_full = datasets.ImageFolder(params['data_dir'], transform=train_transforms)
val_dataset_full = datasets.ImageFolder(params['data_dir'], transform=val_transforms)
train_dataset, _ = random_split(train_dataset_full, [train_size, len(train_dataset_full) - train_size])
val_dataset, _ = random_split(val_dataset_full, [val_size, len(val_dataset_full) - val_size])

# ------------------------------
# Check device
# ------------------------------
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f'Using device: {device}')

# ------------------------------
# Utility Functions
# ------------------------------
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20, experiment_name='experiment', early_stopping=False, patience=5, l1_lambda=0.0):
    results = []
    total_time_start = time.time()
    model = model.to(device)
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_wts = None

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # L1 Regularization
            if l1_lambda > 0.0:
                l1_loss = sum(torch.sum(torch.abs(param)) for param in model.parameters())
                loss += l1_lambda * l1_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.float() / len(train_loader.dataset)
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        results.append({'epoch': epoch+1, 'phase': 'train', 'loss': epoch_loss, 'acc': epoch_acc.item()})

        # Validation phase
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_val_loss = running_loss / len(val_loader.dataset)
        epoch_val_acc = running_corrects.float() / len(val_loader.dataset)
        print(f'Val Loss: {epoch_val_loss:.4f} Acc: {epoch_val_acc:.4f}\n')
        results.append({'epoch': epoch+1, 'phase': 'val', 'loss': epoch_val_loss, 'acc': epoch_val_acc.item()})

        # Check for improvement
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_wts = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # Early Stopping
        if early_stopping and epochs_no_improve >= patience:
            print(f'Early stopping triggered after {epoch+1} epochs.')
            break

        epoch_time = time.time() - epoch_start_time
        print(f'Epoch {epoch+1} completed in {int(epoch_time // 60)}m {int(epoch_time % 60)}s\n')

    # Load best model weights
    if best_model_wts:
        model.load_state_dict(best_model_wts)

    # Total training time
    total_time = time.time() - total_time_start
    print(f"Total training time: {int(total_time // 60)}m {int(total_time % 60)}s")

    # Save log
    log_path = os.path.join(params['save_dir'], f'{experiment_name}_log.txt')
    with open(log_path, 'w') as f:
        for entry in results:
            f.write(f"Epoch {entry['epoch']} {entry['phase']} Loss: {entry['loss']:.4f} Acc: {entry['acc']:.4f}\n")
    print(f"Training log saved to {log_path}\n")

    return model, results

def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)
    accuracy = 100 * correct.float() / total
    print(f'Validation Accuracy: {accuracy:.2f}%')
    return accuracy.item(), all_labels, all_preds

def plot_metrics(results, experiment_name):
    epochs = [entry['epoch'] for entry in results if entry['phase'] == 'train']
    train_loss = [entry['loss'] for entry in results if entry['phase'] == 'train']
    val_loss = [entry['loss'] for entry in results if entry['phase'] == 'val']
    train_acc = [entry['acc'] for entry in results if entry['phase'] == 'train']
    val_acc = [entry['acc'] for entry in results if entry['phase'] == 'val']

    # Plot Loss
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_loss, label='Train Loss', marker='o')
    plt.plot(epochs, val_loss, label='Val Loss', marker='x')
    plt.title(f'{experiment_name} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(params['save_dir'], f'{experiment_name}_Loss.png'))
    plt.close()

    # Plot Accuracy
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_acc, label='Train Accuracy', marker='o')
    plt.plot(epochs, val_acc, label='Val Accuracy', marker='x')
    plt.title(f'{experiment_name} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(params['save_dir'], f'{experiment_name}_Accuracy.png'))
    plt.close()

def plot_confusion_matrix(cm, classes, experiment_name):
    plt.figure(figsize=(20, 18))
    sns.heatmap(cm, annot=False, fmt="d", cmap="Blues",
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix - {experiment_name}', fontsize=20)
    plt.xlabel('Predicted Labels', fontsize=16)
    plt.ylabel('True Labels', fontsize=16)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(os.path.join(params['save_dir'], f'{experiment_name}_Confusion_Matrix.png'))
    plt.close()

# ------------------------------
# Model Setup
# ------------------------------
experiment_config = {
    'experiment_name': 'ResNet18_FineTuning',
    'architecture': 'resnet18',
    'dropout': True,
    'dropout_rate': params['dropout_rate'],
    'fine_tune': True
}

# Load ResNet-18
model = models.resnet18(pretrained=True)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Fine-Tune: Unfreeze the last layer
for param in model.layer4.parameters():
    param.requires_grad = True

# Modify the final fully connected layer
in_features = model.fc.in_features
if experiment_config['dropout']:
    model.fc = nn.Sequential(
        nn.Linear(in_features, 512),
        nn.ReLU(),
        nn.Dropout(experiment_config['dropout_rate']),
        nn.Linear(512, params['num_classes'])
    )
else:
    model.fc = nn.Linear(in_features, params['num_classes'])

# ------------------------------
# Define Criterion and Optimizer
# ------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=params['learning_rate'],
    weight_decay=params['weight_decay']
)

# ------------------------------
# Create DataLoaders
# ------------------------------
train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)

# ------------------------------
# Train the Model
# ------------------------------
model, training_results = train_model(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    num_epochs=params['num_epochs'],
    experiment_name=experiment_config['experiment_name'],
    early_stopping=False,
    patience=5,
    l1_lambda=0.0
)

# ------------------------------
# Evaluate the Model
# ------------------------------
accuracy, all_labels, all_preds = evaluate_model(model, val_loader)

# Save the model
model_save_path = os.path.join(params['save_dir'], f"{experiment_config['experiment_name']}.pth")
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}\n")

# ------------------------------
# Plot Metrics
# ------------------------------
plot_metrics(training_results, experiment_config['experiment_name'])

# ------------------------------
# Confusion Matrix
# ------------------------------
cm = confusion_matrix(all_labels, all_preds)
plot_confusion_matrix(cm, class_names, experiment_config['experiment_name'])

print("Training and evaluation completed. All figures and logs are saved in 'Figure Best 18 100pct'.")

Number of classes: 397
Using device: mps


/Users/ryan/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ryan/myenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/20
----------
Train Loss: 3.4266 Acc: 0.2786
Val Loss: 1.8258 Acc: 0.5277

Epoch 1 completed in 26m 40s

Epoch 2/20
----------
Train Loss: 2.4017 Acc: 0.4227
Val Loss: 1.4578 Acc: 0.6063

Epoch 2 completed in 28m 4s

Epoch 3/20
----------
Train Loss: 2.1214 Acc: 0.4739
Val Loss: 1.3025 Acc: 0.6473

Epoch 3 completed in 30m 10s

Epoch 4/20
----------
Train Loss: 1.9675 Acc: 0.5081
Val Loss: 1.1670 Acc: 0.6747

Epoch 4 completed in 28m 56s

Epoch 5/20
----------
Train Loss: 1.8535 Acc: 0.5317
Val Loss: 1.1076 Acc: 0.6889

Epoch 5 completed in 29m 57s

Epoch 6/20
----------
Train Loss: 1.7733 Acc: 0.5484
Val Loss: 1.0345 Acc: 0.7092

Epoch 6 completed in 28m 36s

Epoch 7/20
----------
Train Loss: 1.7030 Acc: 0.5629
Val Loss: 0.9927 Acc: 0.7189

Epoch 7 completed in 29m 11s

Epoch 8/20
----------
Train Loss: 1.6460 Acc: 0.5759
Val Loss: 0.9425 Acc: 0.7307

Epoch 8 completed in 29m 41s

Epoch 9/20
----------
Train Loss: 1.6015 Acc: 0.5852
Val Loss: 0.9025 Acc: 0.7441

Epoch 9 comple